# QTNM sensitivities
Notebook designed to test the possibilities of using Cramer-Rao Lower Bound (CRLB) for sensitivity calculations.

S. Jones (04/03/24)

In [ ]:
# Load Numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# Load scipy constants
import scipy.constants as const

from matplotlib import cm, ticker

In [ ]:
# Make the default figure size larger
plt.rcParams['figure.figsize'] = (8, 5)
# Allow LateX in the labels
plt.rcParams['text.usetex'] = True
'''
mpl.use("pgf")
mpl.rcParams.update({"pgf.texsystem": "pdflatex",
                     'font.family': 'serif',
                     'text.usetex': True,
                     'pgf.rcfonts': False,})
''' 
# Set the default font size
plt.rcParams.update({'font.size': 14})

In [ ]:
# Define a few basic values
atomicTXSec = 9e-23 # Atomic cross section in m^2
molTXSec = 3.4e-22 # Molecular cross section in m^2
dalton = const.physical_constants['atomic mass constant'][0] # Dalton's number in kg
year = 365.25 * 24 * 60 * 60 # Year in seconds
BF1eV = 2e-13 # Branching fraction of T to last eV
TLifetime = 560975924.0 # Lifetime of T in seconds
TMassDa = 3.01604928 # Mass of T in Da
sigmaFSD = 0.4 # eV standard deviation of the FSD
decayConstant = 1.784e-9 # Decay constant in s^-1

For a given tritium number density $n$, we have a mean free path given by
$$
\lambda = (\sigma n)^{-1} \, .
$$
For a particle travelling with speed, $v$, this gives a mean free time of 
$$
\tau = \lambda / v \, .
$$

In [ ]:
# Write function to calculate the mean free path as a function of number density
def meanFreePath(numberDensity, isAtomic=True):
    if isAtomic:
        return 1 / (numberDensity * atomicTXSec)
    else:
        return 1 / (numberDensity * molTXSec)

# Write function to calculate the mean free time as a function of number density and speed
def meanFreeTime(numberDensity, speed, isAtomic=True):
    return meanFreePath(numberDensity, isAtomic) / speed

In [ ]:
# Make some plots showing the mean free path and time as a function of number density for electrons with a kinetic energy of 18.6 keV
# The number densities should be between 1e16 m^-3 and 1e20 m^-3
electronKE = 18.6e3  # Endpoint electron energyu in eV
# Calculate relativistic speed of the electrons
electronSpeed = const.c * np.sqrt(1 - (1 / (1 + (electronKE * const.e / const.m_e / const.c**2))**2))
electronGamma = 1 / np.sqrt(1 - (electronSpeed / const.c)**2)
print('Endpoint electron speed = ', electronSpeed, "m/s, beta = ", electronSpeed / const.c)
print('Electron gamma = ', electronGamma)
numberDensity = np.logspace(16, 20, 100) # Number densities in m^-3
meanFreePathValues = meanFreePath(numberDensity)
meanFreeTimeValues = meanFreeTime(numberDensity, electronSpeed)

# Plot mean free time against density with a log scale
plt.figure()
plt.loglog(numberDensity / 1e6, meanFreeTimeValues * 1e6)
plt.xlabel('Number density [cm$^{-3}$]') 
plt.ylabel('Mean free time [$\mu$s]')
plt.title('Mean free time for 18.6 keV electrons')
plt.grid()
plt.show()
#plt.savefig('meanFreeTime.pgf')

# Now plot mean free path against density with a log scale
plt.figure()
plt.loglog(numberDensity / 1e6, meanFreePathValues)
plt.xlabel('Number density [cm$^{-3}$]')
plt.ylabel('Mean free path [m]')
plt.title('Mean free path for 18.6 keV electrons')
plt.grid()
plt.show()
#plt.savefig('meanFreePath.pgf')

## Cramer-Rao Lower Bound
A complex chirping signal sampled at $N$ discrete times, $t_n$, may be be represented by the signal vector $\vec{s} = \left[s_0, s_1,..., s_{N-1}\right]$ where
$$
  s_n = A \exp\left[i \left( \omega_0 t_n + c t_n^2 + \phi_0 \right) \right] \, .
$$
Calculating the Cramer-Rao lower bound for $\omega_0$ gives the following result
$$
  \textrm{var}\left(\hat{\omega_0}\right) \geq \frac{12\sigma^2 (2N-1)(8N-11)}{A^2 N(N^4 - 5N^2 + 4)T^2} \, ,
$$
which for large $N$ (and taking a measurement time $t = NT$ and a sample rate $f_s$) maybe expressed as
$$
  \textrm{var}\left(\hat{\omega_0}\right) \gtrsim \frac{192\sigma^2}{A^2 t^3 f_s} \, .
$$
This can be converted into the CRLB for $f_0$ to give
$$
  \textrm{std}\left(\hat{f_0}\right) \gtrsim \frac{4\sqrt{3}\sigma}{A \pi t^{3/2} f_s^{1/2}} \, .
$$


In [ ]:
def f0StdDev(A, noiseSigma, t, f_s):
  '''Calculate the standard deviation of the start frequency

  A: Amplitude of the signal
  noiseSigma: Standard deviation of the noise
  t: Duration of the signal
  f_s: Sampling frequency
  '''
  return noiseSigma * 4.0 * np.sqrt(3) / (A * np.pi * np.sqrt(f_s) * t**1.5)

In [ ]:
def f0(B, Ek, mass):
  '''Calculate the cycltron frequency
  
  B: Magnetic field strength in Tesla
  Ek: Kinetic energy in eV 
  mass: Mass in kg
  '''
  return const.e * B / (2 * np.pi) / (mass + Ek * const.e / const.c**2)

In [ ]:
def larmorPower(B, Ek, mass):
  '''Calculate free space radiation power for a particle in a magnetic field in Watts

  B: Magnetic field strength in Tesla
  Ek: Kinetic energy of the particle in eV
  mass: Mass of the particle in kg
  '''
  f0 = const.e * B / (2 * np.pi * mass)
  gamma = 1 + Ek * const.e / (mass * const.c**2)
  beta = np.sqrt(1 - 1 / gamma**2)

  return 2 * np.pi * const.e**2 * f0**2 * beta**2 / (3 * const.epsilon_0 * const.c * (1 - beta**2))

In [ ]:
# Larmor power for an endpoint electron in a 1 T magnetic field
print('Larmor power for an endpoint electron in a 1 T magnetic field = ', larmorPower(1, electronKE, const.m_e), 'W')

In [ ]:
# Define a few parameters for our simulation
collectionEff = 0.1 # Power collection efficiency of the detector
detSampleRate = 1e9 # Sample rate of the detector in Hz
detNoiseTemp = 5.0 # Noise temperature of the detector in K
detBandwidth = detSampleRate / 2 # Bandwidth of the detector in Hz
noiseSigma = np.sqrt(const.k * detNoiseTemp * detBandwidth) # Standard deviation of the noise thermal noise

Take our electron measurement time to be the mean free time at a given density.
Our signal power is the free space power, $P_{\text{Larmor}}$,  multiplied by the collection efficiency, $\epsilon_P$.
Therefore, we can express the amplitude of our signal in these terms:
$$
  \frac{A^2}{2} = P_{\text{Larmor}} \, \epsilon_P \, .
$$
This gives us an expression for $A$ of 
$$
  A = \sqrt{2 P_{\text{Larmor}} \, \epsilon_P} \, .
$$

In [ ]:
# Create a 2D plot of the standard deviation on the start frequency estimator 
# as a function of the magnetic field strength and gas number density

# Generate magnetic field strengths from 10^-3 to 10 T
B = np.logspace(-3, 1, 100)
# Generate mesh of B and number density
BV, numberDensityV = np.meshgrid(B, numberDensity)
# Now evaluate the standard deviation of the start frequency estimator at each grid point
f0StdDevV = f0StdDev(np.sqrt(2 * larmorPower(BV, electronKE, const.m_e) * collectionEff), noiseSigma, meanFreeTime(numberDensityV, electronSpeed), detSampleRate)
# Create some suitable logarithmic contours
levsExp = np.arange(np.floor(np.log10(np.min(f0StdDevV))), np.ceil(np.log10(np.max(f0StdDevV))+1), 1)
print('Min, max frequency standard deviation = ', np.min(f0StdDevV), np.max(f0StdDevV))
print('Min, max exponential = ', np.min(levsExp), np.max(levsExp))
levs = np.power(10, levsExp)
print(levs)

# Plot the result
fig, ax = plt.subplots()
cs = ax.contourf(B, numberDensity / 1e6, f0StdDevV, levs, norm=mpl.colors.LogNorm()) 
cbar = fig.colorbar(cs, label='$\\rm{std}\left(\hat{f_0}\\right)$ [Hz]')
plt.title("T = {} K, $\epsilon_P$ = {}".format(detNoiseTemp, collectionEff))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Magnetic field strength [T]')
plt.ylabel('Number density [cm$^{-3}$]')
plt.show()
# plt.savefig('f0StdDev.pgf')

In [ ]:
# Now plot the fractional standard deviation on the start frequency estimator
fig, ax = plt.subplots()
f0V = f0(BV, electronKE, const.m_e)

levsExp = np.arange(np.floor(np.log10(np.min(f0StdDevV/f0V))), np.ceil(np.log10(np.max(f0StdDevV/f0V))+1), 1)
print('Min, max frequency standard deviation = ', np.min(f0StdDevV), np.max(f0StdDevV))
print('Min, max exponential = ', np.min(levsExp), np.max(levsExp))
levs = np.power(10, levsExp)

cs = ax.contourf(B, numberDensity / 1e6, f0StdDevV / f0V, levs, norm=mpl.colors.LogNorm())
cbar = fig.colorbar(cs, label='$\\textrm{std}\left(\hat{f_0}\\right) / f_0$')
plt.title("T = {} K, $\epsilon_P$ = {}".format(detNoiseTemp, collectionEff))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Magnetic field strength [T]')
plt.ylabel('Number density [cm$^{-3}$]')
plt.show()
# plt.savefig('f0StdDevFrac.pgf')

The relation between the uncertainty on a frequency measurement and the uncertainty on an energy measurement is given by
$$
\frac{\Delta E}{E} = \frac{\gamma}{\gamma - 1} \frac{\Delta f}{f} \, .
$$
For an endpoint electron, $\gamma = 1.036$.

In [ ]:
# Finally, plot the fractional standard deviation on the energy estimator
fig, ax = plt.subplots()
fracEV = (electronGamma / (electronGamma - 1)) * f0StdDevV / f0V
levsExp = np.arange(np.floor(np.log10(np.min(fracEV))), np.ceil(np.log10(np.max(fracEV))+1), 1)
print('Min, max fractional energy standard deviation = ', np.min(fracEV), np.max(fracEV))
print('Min, max exponential = ', np.min(levsExp), np.max(levsExp))
levs = np.power(10, levsExp)
cs = ax.contourf(B, numberDensity / 1e6, fracEV, levs, norm=mpl.colors.LogNorm()) 
cbar = fig.colorbar(cs, label='$\\textrm{std}\left(\hat{E_0}\\right) / E_0$')
plt.title("T = {} K, $\epsilon_P$ = {}".format(detNoiseTemp, collectionEff))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Magnetic field strength [T]')
plt.ylabel('Number density [cm$^{-3}$]')
plt.show()
# plt.savefig('E0StdDevFrac.pgf')

If we say that we only reconstruct electrons with a minimum track length, $t_{\text{min}}$, then for a given mean free time, $\tau$, the fraction of electrons we will reconstruct is
$$
\epsilon_{\text{rec}} = e^{-t_{\text{min}} / \tau} \, .
$$
Therefore, if our we set $t_{\text{min}} = \tau$, we will only reconstruct $\approx 0.37$ of all electrons.
If we want to reconstruct 75% of all available electrons we require that $t_{\text{min}} = 0.288\tau$.

In [ ]:
def recoFrac(t_min, n, isAtomic=True):
  '''Calculate the fraction of tracks that will be reco'd for a given minimum time and number density
  
  t_min: Minimum time in seconds
  n: Number density in m^-3
  '''
  tau = meanFreeTime(n, electronSpeed, isAtomic)
  return np.exp(-t_min / tau)

In [ ]:
# Now remake the fractional energy resolution plot with a minimum track time such that we reco 75% of the tracks
desiredRecoFrac = 0.75
measurementTimeV = meanFreeTime(numberDensityV, electronSpeed) * -np.log(desiredRecoFrac)
# Now evaluate the standard deviation of the start frequency estimator at each grid point
f0StdDevV = f0StdDev(np.sqrt(2 * larmorPower(BV, electronKE, const.m_e) * collectionEff), noiseSigma, measurementTimeV, detSampleRate)
# We've already calculated f0V
fracEV = (electronGamma / (electronGamma - 1)) * f0StdDevV / f0V

In [ ]:
# Plot fractional energy resolution as a function of number density and magnetic field strength
levsExp = np.arange(np.floor(np.log10(np.min(fracEV))), np.ceil(np.log10(np.max(fracEV))+1), 1)
print('Min, max fractional energy standard deviation = ', np.min(fracEV), np.max(fracEV))
print('Min, max exponential = ', np.min(levsExp), np.max(levsExp))
levs = np.power(10, levsExp)

customLevsExp = np.array([np.floor(np.log10(np.min(fracEV))), -6.0, np.ceil(np.log10(np.max(fracEV))+1)])
customLevs = np.power(10, customLevsExp)

fig, ax = plt.subplots()
cs = ax.contourf(B, numberDensity / 1e6, fracEV, customLevs, norm='log') 
cbar = fig.colorbar(cs, label='$\\textrm{std}\left(\hat{E_0}\\right) / E_0$')
plt.title("T = {} K, $\epsilon_P$ = {}, $\epsilon_r$ = {}".format(detNoiseTemp, collectionEff, desiredRecoFrac))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Magnetic field strength [T]')
plt.ylabel('Number density [cm$^{-3}$]')
plt.grid()
plt.show()
# plt.savefig('E0StdDevFrac_75pcReco.pgf')

## Sensitivity calculations
Define an experiment with the following parameters:
- Active volume, $V$
- Tritium atom number density, $n$
- Magnetic field strength, $B$ and uncertainty $\sigma_B$
- Sample rate, $f_s$
- System noise temperature, $T_{\text{noise}}$
- Atom velocity spread, $\Delta v$
- Trap depth, $\Delta B$
- Minimum track measurement time, $t_{\text{min}}$
- An experimental running time, $t_{\text{live}}$
- A background rate, $b$
- A power collection efficiency, $\epsilon_P$

In [ ]:
# Create function to calculate the analytical sensitivity of an experiment
def experimentSigmaMSq(V, n, B, sigmaB, f_s, T_noise, deltaV, deltaB, t_min, 
                       t_live, b, ep_P, isAtomic: bool = True) -> float:
  '''Function to calculate analytical sensitivity of a CRES experiment

  V: Volume of the experiment in m^3
  n: Number density of the atomic T in m^-3
  B: Magnetic field strength in Tesla
  sigmaB: Standard deviation of the magnetic field in Tesla
  f_s: Sampling frequency in Hz
  T_noise: Noise temperature of the detector in K
  deltaV: Atom velocity spread in m/s
  deltaB: Magnetic trap depth in Tesla
  t_min: Minimum track time in seconds
  t_live: Live time of the experiment in seconds
  b: Background rate in eV^-1 s^-1
  '''

  effectiveAtomTemp = TMassDa * const.m_u * deltaV**2 / (3 * const.k) # Effective temperature of the atoms in K
  trackRecoFrac = recoFrac(t_min, n, isAtomic) # Fraction of tracks that will be reco'd
  # Calculate fraction of pitch angles that are trapped
  dThetaMax = np.arccos(np.sqrt(B / (deltaB + B)))
  trapFrac = np.sin(dThetaMax) # Trapped fraction
  r = trackRecoFrac * trapFrac * n * V * BF1eV / TLifetime # Rate of T decays in last eV
  # Calculate the standard variation of the noise
  noiseSigma = np.sqrt(const.k * T_noise * f_s / 2.0)

  # Calculate the resolution contribution from the magnetic field
  stdE_BField = electronGamma / (electronGamma - 1) * electronKE * sigmaB / B
  # Calculate the resolution contribution from the atom velocity spread
  stdE_AtomT = electronGamma / (electronGamma - 1) * electronKE * np.sqrt(const.k * effectiveAtomTemp / (TMassDa * const.m_u * const.c**2))
  # Now calculate the resolution component from the CRLB
  PColl = larmorPower(B, electronKE, const.m_e) * ep_P
  stdE_CRLB = electronGamma / (electronGamma - 1) * electronKE * f0StdDev(np.sqrt(2 * PColl), noiseSigma, t_min, f_s) / f0(B, electronKE, const.m_e)

  # Combine the resolution components to get an optimal energy interval
  if isAtomic:
    deltaE = np.sqrt(b / r + 8.0 * np.log(2) * (stdE_AtomT**2 + stdE_BField**2 + stdE_CRLB**2))  # eV
    # Need to also calculate resolution contributions to systematic uncertainty
    systUnc = 4.0 * np.sqrt(0.01**2 * (stdE_AtomT**4 + stdE_BField**4 + stdE_CRLB**4)) # eV^2
  else:
    deltaE = np.sqrt(b / r + 8.0 * np.log(2) * (stdE_AtomT**2 + stdE_BField**2 + stdE_CRLB**2 + sigmaFSD**2))  # eV
    # Need to also calculate resolution contributions to systematic uncertainty
    systUnc = 4.0 * np.sqrt(0.01**2 * (stdE_AtomT**4 + stdE_BField**4 + stdE_CRLB**4 + sigmaFSD**4)) # eV^2

  # Calculate the statistical uncertainty
  statUncert = 2.0 / (3.0 * r * t_live) * np.sqrt(r * t_live * deltaE + b * t_live / deltaE) # eV^2

  return np.sqrt(statUncert**2 + systUnc**2)

### 1 T experiment
Experimental parameters are as follows:
- $V = 10~\text{m}^{3}$
- Tritium atom number density, $n = 10^{12}~\text{cm}^{-3}$
- $B = 1~\text{T}$ and $\sigma_B / B = 10^{-6}$
- $f_s = 1~\text{GHz}$
- $T_{\text{noise}} = 5~\text{K}$
- $\Delta v = 7~\text{m/s}$
- $\Delta B$ selected to trap 10% of electrons
- $t_{\text{min}}$ selected to reconstruct 0.75 of all electrons
- $b = 10^{-6}~\text{eV}^{-1} s^{-1}$
- $\epsilon_P = 0.1$
- Plot as a function of $t_{\text{live}}$

In [ ]:
runTimesYears = np.logspace(-4, 3, 100) # Run times in years
runTimesSeconds = runTimesYears * year  # Run times in seconds
# At a 1T field, what trap depth do we require to trap 10% of electrons
BBkg = 1.0 # Tesla
desiredTrapFrac = 0.1
trapDepth = BBkg / (np.cos(np.arcsin(desiredTrapFrac)))**2 - BBkg # Tesla
print('Required trap depth = ', trapDepth * 1e3, 'mT')

# Define other experimental considerations
volume = 10 # m^3
atomNumberDensity = 1e18 # m^-3
sigmaB_1ppm = 1e-6 * BBkg # Standard deviation of the magnetic field in Tesla
Tnoise = 5.0 # Noise temperature of the detector in K
deltaV = 5.0 # Atom velocity spread in m/s
detSampleRate = 1e9 # Sample rate of the detector in Hz
bkgRate = 1e-6 # Background rate in eV^-1 s^-1
epsilonP = 0.1 # Power collection efficiency of the detector
t_min = meanFreeTime(atomNumberDensity, electronSpeed) * -np.log(desiredRecoFrac) # Minimum track time in seconds

# Now calculate the analytical sensitivity for a range of run times
sigmaM2 = experimentSigmaMSq(volume, atomNumberDensity, BBkg, sigmaB_1ppm, 
                             detSampleRate, Tnoise, deltaV, trapDepth, t_min, 
                             runTimesSeconds, bkgRate, epsilonP)
mBeta90CL = np.sqrt(1.28 * sigmaM2) # 90% CL sensitivity on the neutrino mass

# Plot the result
plt.figure()
plt.loglog(runTimesYears, mBeta90CL)
plt.xlabel('Run time [years]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Sensitivity of a $1~\\rm{T}$, $10~\\rm{m}^3$ CRES experiment') 
plt.grid()
plt.show()

In [ ]:
# What about plotting an 'exposure' against the sensitivity?
exposuresYears = np.logspace(-4, 7, 200) # Exposure in m^3 years
exposuresSeconds = exposuresYears * year # Exposure in m^3 seconds
totalEfficiency = desiredTrapFrac * desiredRecoFrac
print('Total efficiency = ', totalEfficiency)

# Now calculate the analytical sensitivity for a range of exposures
oneRunTime = 2.0 * year  # 2 year run time in seconds
# Check what this looks like with and without a magnetic field uncertainty
sigmaB_100ppb = 100e-9 * BBkg # Tesla
sigmaM2 = experimentSigmaMSq(exposuresSeconds / (oneRunTime * totalEfficiency), 
                             atomNumberDensity, BBkg, sigmaB_1ppm, detSampleRate, 
                             Tnoise, deltaV, trapDepth, t_min, oneRunTime, 
                             bkgRate, epsilonP)

sigmaM2_Mol = experimentSigmaMSq(exposuresSeconds / (oneRunTime * totalEfficiency), 
                                 atomNumberDensity, BBkg, sigmaB_1ppm, detSampleRate, 
                                 Tnoise, deltaV, trapDepth, t_min, oneRunTime, 
                                 bkgRate, epsilonP, False)

sigmaM2_100ppb = experimentSigmaMSq(exposuresSeconds / (oneRunTime * totalEfficiency), 
                                    atomNumberDensity, BBkg, sigmaB_100ppb, detSampleRate, 
                                    Tnoise, deltaV, trapDepth, t_min, oneRunTime, 
                                    bkgRate, epsilonP)

Tnoise_40K = 180.0 # Noise temperature of the detector in K
sigmaM2_Noisy = experimentSigmaMSq(exposuresSeconds / (oneRunTime * totalEfficiency), 
                                   atomNumberDensity, BBkg, sigmaB_100ppb, detSampleRate, 
                                   Tnoise_40K, deltaV, trapDepth, t_min, oneRunTime, 
                                   bkgRate, epsilonP)

# Plot the result
plt.figure()
plt.loglog(exposuresYears, np.sqrt(1.28 * sigmaM2), 
           label='$\\frac{\sigma_B}{B} = 10^{-6}$, $\\rm{T}$')
plt.loglog(exposuresYears, np.sqrt(1.28 * sigmaM2_Mol), 
           label='$\\frac{\sigma_B}{B} = 10^{-6}$, $\\rm{T}_2$')
plt.loglog(exposuresYears, np.sqrt(1.28 * sigmaM2_100ppb), 
           label='$\\frac{\sigma_B}{B} = 10^{-7}$, $\\rm{T}$')
plt.loglog(exposuresYears, np.sqrt(1.28 * sigmaM2_Noisy), 
           label='$\\frac{\sigma_B}{B} = 10^{-7}$, $\\rm{T}$, $T_{\\rm{noise}} = 180~\\rm{K}$')
plt.xlabel('Efficiency $\\times$ Volume $\\times$ Time  [$\\rm{m}^3$ years]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Sensitivity of a $1~\\rm{T}$, $10~\\rm{m}^3$ CRES experiment')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Plot some sensitivities as a function of density
experimentVolume = 50.0 # m^3
oneRunTime = 10.0 * year  # 10 year run time in seconds
numberDensities = 0.5 * np.logspace(16, 20, 200) # Number densities in m^-3
minimumTimes = meanFreeTime(numberDensities, electronSpeed) * -np.log(desiredRecoFrac) # Minimum track time in seconds
sigmaM2_1T = experimentSigmaMSq(experimentVolume, numberDensities, 1.0, 
                                sigmaB_100ppb, detSampleRate, Tnoise, deltaV, 
                                trapDepth, minimumTimes, oneRunTime, bkgRate, 
                                epsilonP)

bField_100mT = 0.1 # Tesla
trapDepth_100mT = bField_100mT / (np.cos(np.arcsin(desiredTrapFrac)))**2 - bField_100mT # Tesla
sigmaM2_100mT = experimentSigmaMSq(experimentVolume, numberDensities, 0.5, 
                                   bField_100mT * 1e-7, detSampleRate, Tnoise, deltaV, 
                                   trapDepth_100mT, minimumTimes, oneRunTime, 
                                   bkgRate, epsilonP)

plt.figure()
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_1T), 
           label='$B = 1~\\rm{T}$')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_100mT), 
           label='$B = 0.5~\\rm{T}$')
plt.xlabel('Number density [cm$^{-3}$]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Sensitivity of a $50~\\rm{m}^3$, 10 year CRES experiment')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# For a given run time plot the sensitivity as a function of volume
# Should allow us to show the effect of the number density
runTime = 10.0 * year  # 10 year run time in seconds
volumes = np.logspace(-1, 4, 200) # Volumes in m^3
sigmaM2_1T = experimentSigmaMSq(volumes, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                detSampleRate, Tnoise, deltaV, trapDepth, t_min, 
                                runTime, bkgRate, epsilonP)

# High number density experiment
highNumberDensity = 5e18 # m^-3
t_min_highDensity = meanFreeTime(highNumberDensity, electronSpeed) * -np.log(desiredRecoFrac) # Minimum track time in seconds
sigmaM2_1T_highDensity = experimentSigmaMSq(volumes, highNumberDensity, BBkg, sigmaB_100ppb, 
                                detSampleRate, Tnoise, deltaV, trapDepth, t_min_highDensity, 
                                runTime, bkgRate, epsilonP)

bField_500mT = 0.5 # Tesla
sigmaM2_500mT = experimentSigmaMSq(volumes, atomNumberDensity, bField_500mT, 1e-7 * bField_500mT, 
                                   detSampleRate, Tnoise, deltaV, trapDepth, t_min, 
                                   runTime, bkgRate, epsilonP)

plt.figure()
plt.loglog(volumes, np.sqrt(1.28 * sigmaM2_1T_highDensity), 
           label='$B = 1~\\rm{T}$, $n = 5 \\times 10^{12}~\\rm{cm}^{-3}$')
plt.loglog(volumes, np.sqrt(1.28 * sigmaM2_1T), 
           label='$B = 1~\\rm{T}$, $n = 10^{12}~\\rm{cm}^{-3}$')
plt.loglog(volumes, np.sqrt(1.28 * sigmaM2_500mT), 
           label='$B = 0.5~\\rm{T}$, $n = 10^{12}~\\rm{cm}^{-3}$')
plt.xlabel('Volume [m$^3$]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Sensitivity of a 10 year CRES experiment')
plt.grid()
plt.legend()
plt.show()
# plt.savefig('densityPlot.pgf')

# This plot nicely shows how higher densities degrade your sensitivity

Project 8 literature gives energy resolution component as
$$
  \sigma_E = \frac{E}{\gamma - 1} \frac{\beta c \sigma_0 n}{2\pi f_c} \, .
$$
How does this compare to the limits from the CRLB.

In [ ]:
# Make a plot of the ratio of the two resolution components as a function field
# strength and number density
# We already have the mesh of B and number density and the fractional standard 
# deviation on the energy estimator
# Calculate the resolution component via the other method
def sigmaE_P8(B, n, isAtomic=True):
  '''Calculate the resolution component from the CRLB

  B: Magnetic field strength in Tesla
  n: Number density in m^-3
  isAtomic: Boolean indicating whether the experiment is T or T2
  '''
  f_c = f0(B, electronKE, const.m_e) # Cyclotron frequency in Hz

  if isAtomic:
    return electronGamma / (electronGamma - 1) * electronKE * electronSpeed * n * atomicTXSec / (2 * np.pi * f_c)
  else:
    return electronGamma / (electronGamma - 1) * electronKE * electronSpeed * n * molTXSec / (2 * np.pi * f_c)
  
sigmaEFrac_P8V = sigmaE_P8(BV, numberDensityV, True) / electronKE

# Plot the result
fig, ax = plt.subplots()
levsExp = np.arange(np.floor(np.log10(np.min(sigmaEFrac_P8V / fracEV))), 
                    np.ceil(np.log10(np.max(sigmaEFrac_P8V / fracEV))), 0.5)
levs = np.power(10, levsExp)
cs = ax.contourf(B, numberDensity / 1e6, sigmaEFrac_P8V / fracEV, levs, norm='log')
cbar = fig.colorbar(cs, label='Project 8 / QTNM')
plt.title("Comparison of $\sigma_E$ from scattering and CRLB")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Magnetic field strength [T]')
plt.ylabel('Number density [cm$^{-3}$]')
plt.show()

In [ ]:
# Create function for calculating required observation time to reach a given sensitivity
def ObsTimeFromEnergyRes(fractionalRes, B, T_noise, f_s, ep_P):
  '''Calculate the observation time required to reach a given energy resolution

  fractionalRes: Fractional energy resolution desired
  B: Magnetic field strength in Tesla
  T_noise: Noise temperature of the detector in K
  f_s: Sampling frequency in Hz
  ep_P: Power collection efficiency of the detector
  '''
  # Calculate the standard variation of the noise
  noiseSigma = np.sqrt(const.k * T_noise * f_s / 2.0)
  # Calculate the resolution component from the CRLB
  PColl = larmorPower(B, electronKE, const.m_e) * ep_P
  tTemp = (electronGamma / (electronGamma - 1)) * (4.0 * noiseSigma / (np.pi * f0(B, electronKE, const.m_e))) * np.sqrt(3.0 / (2 * f_s * PColl)) / fractionalRes
  return np.power(tTemp, 2.0 / 3.0)

In [ ]:
# Plot required observation time for 1 ppm resolution as a function of B field
TNoise5K = 5.0 # Noise temperature of the detector in K

requiredTimes = ObsTimeFromEnergyRes(1e-6, B, TNoise5K, detSampleRate, 
                                     epsilonP)
fig, ax = plt.subplots()
ax.loglog(B, requiredTimes * 1e6)
ax.set_xlabel('Magnetic field strength [T]')
ax.set_ylabel('Required observation time [$\mu$s]')
ax.set_title('$t_{\\rm{min}}$ for 1 ppm energy resolution, $T_{\\rm{noise}} = 5~\\rm{K}$')
plt.grid()
plt.show()

In [ ]:
# Keeping the observation time constant, how does changing the number density 
# change the sensitvitity?
# We want a sensitivity plot with a constant volume and changing observation time
# Calculate required observation time for 1 ppm E resolution at 1 T
desiredERes = 1e-6
requiredTime = ObsTimeFromEnergyRes(desiredERes, BBkg, TNoise5K, detSampleRate, 
                                    epsilonP)
experimentVolume = 100.0 # m^3
runTimesYears = np.logspace(-3, 3, 100) # Run times in years
# Create array of number densities
numberDensities =  np.logspace(16, 20, 200) # Number densities in m^-3

# Create a few lines
sigmaM2_LowDensity = experimentSigmaMSq(experimentVolume, 1e17, BBkg, 
                                        sigmaB_100ppb, detSampleRate, TNoise5K,
                                        deltaV, trapDepth, requiredTime, 
                                        runTimesYears * year, bkgRate, epsilonP)
sigmaM2_MidDensity = experimentSigmaMSq(experimentVolume, 1e18, BBkg, 
                                        sigmaB_100ppb, detSampleRate, TNoise5K,
                                        deltaV, trapDepth, requiredTime, 
                                        runTimesYears * year, bkgRate, epsilonP)
sigmaM2_HiDensity = experimentSigmaMSq(experimentVolume, 1e19, BBkg, 
                                       sigmaB_100ppb, detSampleRate, TNoise5K,
                                       deltaV, trapDepth, requiredTime, 
                                       runTimesYears * year, bkgRate, epsilonP)
sigmaM2_HiHiDensity = experimentSigmaMSq(experimentVolume, 1e20, BBkg, 
                                         sigmaB_100ppb, detSampleRate, TNoise5K,
                                         deltaV, trapDepth, requiredTime, 
                                         runTimesYears * year, bkgRate, epsilonP)

# Plot these lines together
plt.figure()
plt.loglog(runTimesYears, np.sqrt(1.28 * sigmaM2_LowDensity), 
           label='$n = 10^{11}~\\rm{cm}^{-3}$')
plt.loglog(runTimesYears, np.sqrt(1.28 * sigmaM2_MidDensity), 
           label='$n = 10^{12}~\\rm{cm}^{-3}$')
plt.loglog(runTimesYears, np.sqrt(1.28 * sigmaM2_HiDensity), 
           label='$n = 10^{13}~\\rm{cm}^{-3}$')
plt.loglog(runTimesYears, np.sqrt(1.28 * sigmaM2_HiHiDensity), 
           label='$n = 10^{14}~\\rm{cm}^{-3}$')
plt.xlabel('Run time [years]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Sensitivity of a $100~\\rm{m}^3$ CRES experiment')
plt.grid()
plt.legend()
plt.show()

# What do our 2D plots of number density and run time look like?
# Create a mesh of number density and observation time
runTimesYearsV, numberDensitiesV = np.meshgrid(runTimesYears, numberDensities)
# Now evaluate the sensitivity at each grid point
sigmaM2V = experimentSigmaMSq(experimentVolume, numberDensitiesV, BBkg, 
                              sigmaB_100ppb, detSampleRate, TNoise5K, deltaV, 
                              trapDepth, requiredTime, runTimesYearsV * year, 
                              bkgRate, epsilonP)
# Plot the result
fig, ax = plt.subplots()
levsExp = np.arange(np.floor(np.log10(np.min(np.sqrt(1.28 * sigmaM2V)))), 
                    np.ceil(np.log10(np.max(np.sqrt(1.28 * sigmaM2V)))), 0.5)
levs = np.power(10, levsExp)
cs = ax.contourf(runTimesYears, numberDensities / 1e6, np.sqrt(1.28 * sigmaM2V), 
                 levs, norm='log')
cbar = fig.colorbar(cs, label='90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title("Sensitivity of a $100~\\rm{m}^3$, 1 T CRES experiment, $T = 5~\\rm{K}$")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Run time [years]')
plt.ylabel('Number density [cm$^{-3}$]')
# Find the location of the minimum in the plot and draw a line at that density
minLoc = np.unravel_index(np.argmin(np.sqrt(1.28 * sigmaM2V), axis=None), np.shape(sigmaM2V))
optimumDensity_5K = numberDensities[minLoc[0]]
plt.axhline(y=optimumDensity_5K / 1e6, color='r', linestyle='--')
plt.show()
plt.savefig('densityRuntime_5K.pgf')

# Now evaluate the sensitivity at each grid point
TNoise180K = 180.0 # Noise temperature of the detector in K
requiredTime180K = ObsTimeFromEnergyRes(desiredERes, BBkg, TNoise180K, detSampleRate, epsilonP)
sigmaM2V180K = experimentSigmaMSq(experimentVolume, numberDensitiesV, BBkg, sigmaB_100ppb, detSampleRate, 
                                  TNoise180K, deltaV, trapDepth, requiredTime180K, runTimesYearsV * year, 
                                  bkgRate, epsilonP)

# Plot the result
fig, ax = plt.subplots()
# levsExp = np.arange(np.floor(np.log10(np.min(np.sqrt(1.28 * sigmaM2V180K)))), 
#                     np.ceil(np.log10(np.max(np.sqrt(1.28 * sigmaM2V180K)))), 0.5)
levsExp = np.arange(np.floor(np.log10(np.min(np.sqrt(1.28 * sigmaM2V180K)))), 
                    10, 0.5)
levs = np.power(10, levsExp)
cs = ax.contourf(runTimesYears, numberDensities / 1e6, np.sqrt(1.28 * sigmaM2V180K), 
                 levs, norm='log')
cbar = fig.colorbar(cs, label='90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title("Sensitivity of a $100~\\rm{m}^3$, 1 T CRES experiment, $T = 180~\\rm{K}$")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Run time [years]')
plt.ylabel('Number density [cm$^{-3}$]')
minLoc = np.unravel_index(np.argmin(np.sqrt(1.28 * sigmaM2V180K), axis=None), np.shape(sigmaM2V180K))
optimumDensity_180K = numberDensities[minLoc[0]]
plt.axhline(y=optimumDensity_180K / 1e6, color='r', linestyle='--')
plt.show()
#plt.savefig('densityRuntime_180K.pgf')
print('Optimum density at 5 K = ', optimumDensity_5K)
print('Optimum density at 180 K = ', optimumDensity_180K)

In [ ]:
# Some plots to aid understanding a little bit more
# As we change the density, how does the reconstructed fraction change?
# As we change the density, how does the total event rate change?
recoFractions = recoFrac(requiredTime, numberDensities)
totalEventRatesLasteV = numberDensities * experimentVolume * BF1eV / TLifetime
recoEventRatesLasteV = totalEventRatesLasteV * recoFractions
# Plot these on the same graph with different y-axes
fig, ax1 = plt.subplots()
ax1.set_xlabel('Number density [cm$^{-3}$]')
ax1.set_xscale('log')
ax1.set_ylabel('Reconstructed fraction', color='tab:blue')
ax1.plot(numberDensities / 1e6, recoFractions, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax2 = ax1.twinx()
ax2.set_ylabel('Last eV event rate [s$^{-1}$]', color='tab:red')
ax2.plot(numberDensities / 1e6, totalEventRatesLasteV, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
plt.title('100 m$^3$, 1 T CRES experiment')
plt.show()
# plt.savefig('eventRateDensity.pgf')

# Plot the reconstructed event rate as a function of the density
plt.figure()
plt.loglog(numberDensities / 1e6, recoEventRatesLasteV)
plt.xlabel('Number density [cm$^{-3}$]')
plt.ylabel('Reconstructed last eV event rate [s$^{-1}$]')
plt.title('100 m$^3$, 1 T CRES experiment')
plt.grid()
plt.show()
# plt.savefig('recoEventRate.pgf')

## $\rm{T}_2$ contamination
The endpoint of molecular tritium is $\sim 8~\rm{eV}$ higher than that of atomic tritium, presenting itself as a background.
The rate factor for the decay of trtium is given by
$$
  r_0 = \frac{\lambda}{C_t E_0^3} \, ,
$$
where $\lambda = 1.784 \times 10^{-9}~\rm{s}^{-1}$ is the tritium decay constant.
For atomic tritium $C_t = 0.767$, while for molecular tritium, $C_t = 0.943$.
Therefore, one can plot the overlap of the two decay spectra to get an idea of the overlap.

In [ ]:
# Create function for tritium decay spectrum around the endpoint
def TritiumDecayRate(E, E0, mBeta, isAtomic: bool):
  '''Calculate the tritium decay rate around the endpoint

  E: Energy of the decay in eV
  E0: Endpoint energy in eV
  mBeta: Neutrino mass in eV
  '''
  r0 = 0 # Rate factor
  if isAtomic:
    r0 = decayConstant / (0.767 * E0**3)
  else:
    r0 = decayConstant / (0.943 * E0**3)

  return 3 * r0 * (E0 - E) * np.sqrt((E0 - E)**2 - mBeta**2) * np.heaviside(E0 - E - mBeta, 0)

In [ ]:
# Plot the decay spectra for molecular and atomic tritium
endpointMol = 18.575e3 # eV
endpointAtom = endpointMol - 8.0 # eV
Energies = np.linspace(18560, endpointMol, 400) # Energies in eV
decayRatesMol = TritiumDecayRate(Energies, endpointMol, 0.0, False)
decayRatesAtom = TritiumDecayRate(Energies, endpointAtom, 0.0, True)
# Plot the result
plt.figure()
plt.plot(Energies - endpointMol, decayRatesMol, label='$\\rm{T}_2$')
plt.plot(Energies - endpointMol, decayRatesAtom, label='$\\rm{T}$')
plt.xlabel('$E - E_{0,~\\rm{mol}}$ [eV]')
plt.ylabel('Decay rate [s$^{-1}$ eV$^{-1}$]')
plt.title('Approximate tritium decay spectra')
plt.grid()
plt.legend()
plt.show()

Some background calculation on this subject from Formaggio, de Gouvea and Robertson (2021).
Total rate in the analysis window (with $m_\beta = 0$) becomes
$$
N_{\text{tot}} \simeq r_a t (\Delta E)^3 + bt \Delta E + r_m t \left[\left(E_m + \Delta E\right)^3 - E_m^3\right] \, ,
$$
where $E_m \simeq 8~\text{eV}$ and $r_a$ and $r_m$ are the rates in the last eV of the atomic and molecular respectively.
The statistical uncertainty on $m_\beta^2 then becomes
$$
\sigma_{m_\beta^2} = \frac{2}{3 r_a t} \sqrt{(r_a + r_m)t \Delta E + 3 r_m t E_m + \frac{(b + 3 E_m^2 r_m)t}{\Delta E}} \, ,
$$
and the optimum analysis window then becomes
$$
  \Delta E_{\text{opt}} = \sqrt{\frac{b + 3 E_m^2 r_m}{r_a + r_m}} \, .
$$

In [ ]:
# For the stats only case, how does the optimal energy window change with 
# molecular tritium contamination?
def DeltaEOptMolT(molFrac, nAtoms, b):
  '''Calculate the optimal energy window for a given molecular tritium fraction

  molFrac: Fraction of the tritium that is molecular
  nAtoms: Total number of tritium atoms and molecules in the experiment
  b: Flat background rate in eV^-1 s^-1
  '''
  Em = 8.0 # eV
  E0_mol = 18.575e3 # eV
  E0_atom = E0_mol - Em # eV
  r0_a = decayConstant / (0.767 * E0_atom**3)
  r_a = nAtoms * r0_a * (1.0 - molFrac)
  r0_m = decayConstant / (0.943 * E0_mol**3)
  r_m = nAtoms * r0_m * molFrac
  return np.sqrt((b + 3.0 * Em**2 * r_m) / (r_a + r_m))

In [ ]:
# Generate a range of molecular fractions
molFracs = np.logspace(-6, -1, 100)
flatBkg = 1e-6 # eV^-1 s^-1
nAtoms = 1e20 # Number of tritium atoms and molecules
deltaEVals = DeltaEOptMolT(molFracs, nAtoms, flatBkg)
# Plot the result
plt.figure()
plt.loglog(molFracs, deltaEVals)
plt.xlabel('Molecular tritium fraction')
plt.ylabel('$\Delta E_{\\rm{opt}}$ [eV]')
plt.title('Optimal energy window (stats only)')
plt.grid()
plt.show()
# plt.savefig('deltaEOptMolT.pgf')

In [ ]:
# Create a function for the error on mBeta^2 including the background from 
# molecular tritium contamination
def experimentSigmaMSq_molT(V, n, B, sigmaB, f_s, T_noise, deltaV, deltaB, t_min, 
                            t_live, b, ep_P, molTFrac, resUnc=0.01) -> float:
  '''Function to calculate analytical sensitivity of a CRES experiment

  V: Volume of the experiment in m^3
  n: Number density of the atomic T in m^-3
  B: Magnetic field strength in Tesla
  sigmaB: Standard deviation of the magnetic field in Tesla
  f_s: Sampling frequency in Hz
  T_noise: Noise temperature of the detector in K
  deltaV: Atom velocity spread in m/s
  deltaB: Magnetic trap depth in Tesla
  t_min: Minimum track time in seconds
  t_live: Live time of the experiment in seconds
  b: Background rate in eV^-1 s^-1
  ep_P: Power collection efficiency of the detector
  molTFrac: Fraction of the tritium that is molecular
  resUnc: Fractional uncertainties on the resolution components
  '''

  effectiveAtomTemp = TMassDa * const.m_u * deltaV**2 / (3 * const.k) # Effective temperature of the atoms in K
  trackRecoFrac = recoFrac(t_min, n, True) # Fraction of tracks that will be reco'd
  # Calculate fraction of pitch angles that are trapped
  dThetaMax = np.arccos(np.sqrt(B / (deltaB + B)))
  trapFrac = np.sin(dThetaMax) # Trapped fraction
  efficiency = trackRecoFrac * trapFrac

  # Calculate the atomic and molecular tritium decay rates
  E0Mol = 18.575e3 # eV
  Em = 8.0 # eV
  E0Atom = E0Mol - Em # eV
  r_a = efficiency * n * V * (1.0 - molTFrac) * decayConstant / (0.767 * E0Atom**3)
  r_m = efficiency * n * V * molTFrac * decayConstant / (0.943 * E0Mol**3)

  # Calculate the standard variation of the noise
  noiseSigma = np.sqrt(const.k * T_noise * f_s / 2.0)

  # Calculate the resolution contribution from the magnetic field
  stdE_BField = electronGamma / (electronGamma - 1) * electronKE * sigmaB / B
  # Calculate the resolution contribution from the atom velocity spread
  stdE_AtomT = electronGamma / (electronGamma - 1) * electronKE * np.sqrt(const.k * effectiveAtomTemp / (TMassDa * const.m_u * const.c**2))
  # Now calculate the resolution component from the CRLB
  PColl = larmorPower(B, electronKE, const.m_e) * ep_P
  stdE_CRLB = electronGamma / (electronGamma - 1) * electronKE * f0StdDev(np.sqrt(2 * PColl), noiseSigma, t_min, f_s) / f0(B, electronKE, const.m_e)

  # Combine the resolution components to get an optimal energy interval
  deltaE = np.sqrt((b + 3 * Em**2 * r_m) / (r_a + r_m) + 8.0 * np.log(2) * (stdE_AtomT**2 + stdE_BField**2 + stdE_CRLB**2))  # eV
  # Need to also calculate resolution contributions to systematic uncertainty
  systUnc = 4.0 * np.sqrt(resUnc**2 * (stdE_AtomT**4 + stdE_BField**4 + stdE_CRLB**4)) # eV^2

  # Calculate the statistical uncertainty
  statUncert = 2.0 / (3.0 * r_a * t_live) * np.sqrt(t_live *((r_a + r_m) * deltaE + 3 * r_m * Em + (b + 3 * Em**2 * r_m) / deltaE)) # eV^2

  return np.sqrt(statUncert**2 + systUnc**2)

In [ ]:
# For some reasonable choices of molecular contamination, how does our 
# sensitivity change?
molFrac1ppm = 1e-6
molFrac10ppm = 1e-5
molFrac100ppm = 1e-4
molFrac1ppthousand = 1e-3
molFrac1pphundred = 1e-2

# Basic parameters
experimentVolume = 20 # m^3
atomNumberDensity = 1e18 # m^-3
runTimesYears = np.logspace(-3, 1, 100) # Run times in years
runTimesSeconds = runTimesYears * year  # Run times in seconds

mBeta90CL_1ppmT2 = np.sqrt(1.28 * experimentSigmaMSq_molT(experimentVolume, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                                          detSampleRate, Tnoise, deltaV, trapDepth, t_min, runTimesSeconds, 
                                                          bkgRate, epsilonP, molFrac1ppm))
mBeta90CL_10ppmT2 = np.sqrt(1.28 * experimentSigmaMSq_molT(experimentVolume, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                                           detSampleRate, Tnoise, deltaV, trapDepth, t_min, runTimesSeconds, 
                                                           bkgRate, epsilonP, molFrac10ppm))
mBeta90CL_100ppmT2 = np.sqrt(1.28 * experimentSigmaMSq_molT(experimentVolume, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                                            detSampleRate, Tnoise, deltaV, trapDepth, t_min, runTimesSeconds, 
                                                            bkgRate, epsilonP, molFrac100ppm))
mBeta90CL_1ppthousandT2 = np.sqrt(1.28 * experimentSigmaMSq_molT(experimentVolume, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                                                 detSampleRate, Tnoise, deltaV, trapDepth, t_min, runTimesSeconds, 
                                                                 bkgRate, epsilonP, molFrac1ppthousand))
mBeta90CL_1pphundredT2 = np.sqrt(1.28 * experimentSigmaMSq_molT(experimentVolume, atomNumberDensity, BBkg, sigmaB_100ppb, 
                                                                detSampleRate, Tnoise, deltaV, trapDepth, t_min, runTimesSeconds, 
                                                                bkgRate, epsilonP, molFrac1pphundred))

# Plot these on the same graph
plt.figure()
plt.loglog(runTimesYears, mBeta90CL_1ppmT2, label='$10^{-6}$')
plt.loglog(runTimesYears, mBeta90CL_10ppmT2, label='$10^{-5}$')
plt.loglog(runTimesYears, mBeta90CL_100ppmT2, label='$10^{-4}$')
plt.loglog(runTimesYears, mBeta90CL_1ppthousandT2, label='$10^{-3}$')
plt.loglog(runTimesYears, mBeta90CL_1pphundredT2, label='$10^{-2}$')
plt.xlabel('Run time [years]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.title('Effect of different molecular tritium fractions on sensitivity')
plt.grid()
plt.legend(title='Molecular tritium fraction', ncol=2)
plt.show()

In [ ]:
# Plot things in terms of volume x years, without the efficiency
# Try a few different scenarios and plot them all on the same graph
# We want to show:
# - The need for good magnetic field measurments/knowledge
# - The need for a low system noise temperature
# - The need for a low level of T2 contamination 
# - The trade-off over density

# Run all scenarios with:
# - 1 T magnetic field
# - An atom speed spread of 5 m/s
# - A running time of 7 years
# - Trap depth of 10 mT
# - A flat background of 1e-6 eV^-1 s^-1
# - Power collection fraction of 0.1
# - A measurement time which ensures an energy  resolution of 1 ppm
BField = 1.0 # Tesla
deltaV = 5.0 # m/s
runTimeYears = 7.0  # 7 years
runTimeSeconds = runTimeYears * year  # Run time in seconds
trapDepth = 10.0e-3 # Tesla
bkgRate = 1e-6 # eV^-1 s^-1
powerCollectionEff = 0.1
energyResFrac = 1e-6

exposures = np.logspace(-1, 5, 100) # Exposure in m^3 years

# The base scenario (1) case:
# - Noise temperauture of 180 K
# - Magnetic field uncertainty of 1 ppm
# - 1e-3 T2 contamination
# - 1 x 10^12 cm^-3 number density
TNoise1 = 180.0 # K
sigmaB1 = 1e-6 * BField # Tesla
molFrac1 = 1e-3
numberDensity1 = optimumDensity_180K
t_obs1 = ObsTimeFromEnergyRes(energyResFrac, BField, TNoise1, detSampleRate, powerCollectionEff)
sigmaMBetaSq1 = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity1, BField, sigmaB1, detSampleRate, TNoise1, 
                                        deltaV, trapDepth, t_obs1, runTimeSeconds, bkgRate, powerCollectionEff, molFrac1)
sigmaMBetaSq1_LowUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity1, BField, sigmaB1, detSampleRate, TNoise1, 
                                               deltaV, trapDepth, t_obs1, runTimeSeconds, bkgRate, powerCollectionEff, molFrac1, 0.001)
sigmaMBetaSq1_HighUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity1, BField, sigmaB1, detSampleRate, TNoise1, 
                                               deltaV, trapDepth, t_obs1, runTimeSeconds, bkgRate, powerCollectionEff, molFrac1, 0.1)

# Scenario 2, lowering B field uncertainty:
# - Noise temperauture of 180 K
# - Magnetic field uncertainty of 0.3 ppm
# - 1e-3 T2 contamination
# - 1 x 10^12 cm^-3 number density
TNoise2 = 180.0 # K
sigmaB2 = 0.3e-6 * BField # Tesla
molFrac2 = 1e-3
numberDensity2 = optimumDensity_180K
t_obs2 = ObsTimeFromEnergyRes(energyResFrac, BField, TNoise2, detSampleRate, powerCollectionEff)
sigmaMBetaSq2 = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity2, BField, sigmaB2, detSampleRate, TNoise2, 
                                        deltaV, trapDepth, t_obs2, runTimeSeconds, bkgRate, powerCollectionEff, molFrac2)
sigmaMBetaSq2_LowUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity2, BField, sigmaB2, detSampleRate, TNoise2, 
                                               deltaV, trapDepth, t_obs2, runTimeSeconds, bkgRate, powerCollectionEff, molFrac2, 0.001)
sigmaMBetaSq2_HighUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity2, BField, sigmaB2, detSampleRate, TNoise2, 
                                                deltaV, trapDepth, t_obs2, runTimeSeconds, bkgRate, powerCollectionEff, molFrac2, 0.1)
# Scenario 3, lowering the molecular contamination:
# - Noise temperauture of 180 K
# - Magnetic field uncertainty of 0.3 ppm
# - 1e-6 T2 contamination
# - 1 x 10^12 cm^-3 number density
TNoise3 = 180.0 # K
sigmaB3 = 0.3e-6 * BField # Tesla
molFrac3 = 1e-6
numberDensity3 = optimumDensity_180K
t_obs3 = ObsTimeFromEnergyRes(energyResFrac, BField, TNoise3, detSampleRate, powerCollectionEff)
sigmaMBetaSq3 = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity3, BField, sigmaB3, detSampleRate, TNoise3, 
                                        deltaV, trapDepth, t_obs3, runTimeSeconds, bkgRate, powerCollectionEff, molFrac3)
sigmaMBetaSq3_LowUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity3, BField, sigmaB3, detSampleRate, TNoise3, 
                                               deltaV, trapDepth, t_obs3, runTimeSeconds, bkgRate, powerCollectionEff, molFrac3, 0.001)
sigmaMBetaSq3_HighUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity3, BField, sigmaB3, detSampleRate, TNoise3, 
                                                deltaV, trapDepth, t_obs3, runTimeSeconds, bkgRate, powerCollectionEff, molFrac3, 0.1)

# Scenario 4, lowering the system noise temperature:
# - Noise temperauture of 5 K
# - Magnetic field uncertainty of 0.3 ppm
# - 1e-6 T2 contamination
# - 1.6 x 10^12 cm^-3 number density
TNoise4 = 5.0 # K
sigmaB4 = 0.3e-6 * BField # Tesla
molFrac4 = 1e-6
numberDensity4 = optimumDensity_5K
t_obs4 = ObsTimeFromEnergyRes(energyResFrac, BField, TNoise4, detSampleRate, powerCollectionEff)
sigmaMBetaSq4 = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity4, BField, sigmaB4, detSampleRate, TNoise4, 
                                        deltaV, trapDepth, t_obs4, runTimeSeconds, bkgRate, powerCollectionEff, molFrac4)
sigmaMBetaSq4_LowUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity4, BField, sigmaB4, detSampleRate, TNoise4, 
                                               deltaV, trapDepth, t_obs4, runTimeSeconds, bkgRate, powerCollectionEff, molFrac4, 0.001)
sigmaMBetaSq4_HighUnc = experimentSigmaMSq_molT(exposures / runTimeYears, numberDensity4, BField, sigmaB4, detSampleRate, TNoise4, 
                                                deltaV, trapDepth, t_obs4, runTimeSeconds, bkgRate, powerCollectionEff, molFrac4, 0.1)

# Draw the plots
plt.figure()
'''
plt.loglog(exposures, np.sqrt(1.28 * sigmaMBetaSq1), label='Scenario 1')
plt.loglog(exposures, np.sqrt(1.28 * sigmaMBetaSq2), label='Scenario 2', linestyle='--')
plt.loglog(exposures, np.sqrt(1.28 * sigmaMBetaSq3), label='Scenario 3', linestyle='-.')
plt.loglog(exposures, np.sqrt(1.28 * sigmaMBetaSq4), label='Scenario 4', linestyle=':')
'''
# Plot a band between the high and low uncertainty cases for each scenario
plt.fill_between(exposures, np.sqrt(1.28 * sigmaMBetaSq1_LowUnc), np.sqrt(1.28 * sigmaMBetaSq1_HighUnc), alpha=0.3, label='Scenario 1')
plt.fill_between(exposures, np.sqrt(1.28 * sigmaMBetaSq2_LowUnc), np.sqrt(1.28 * sigmaMBetaSq2_HighUnc), alpha=0.3, label='Scenario 2')
plt.fill_between(exposures, np.sqrt(1.28 * sigmaMBetaSq3_LowUnc), np.sqrt(1.28 * sigmaMBetaSq3_HighUnc), alpha=0.3, label='Scenario 3')
plt.fill_between(exposures, np.sqrt(1.28 * sigmaMBetaSq4_LowUnc), np.sqrt(1.28 * sigmaMBetaSq4_HighUnc), alpha=0.3, label='Scenario 4')
plt.xlabel('Volume $\\times$ Time  [$\\rm{m}^3$ years]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
# Set log scale on x axis and y-axis
plt.xscale('log')
plt.yscale('log')
plt.title('1 T CRES experiment')
plt.grid()
plt.legend(loc='lower left')
plt.xlim([1e-1, 1e5])
#plt.show()
plt.savefig('sensitivityScenarios.pdf')

## Reproducing Frank's plots
Frank looks at two magnetic field scenarios, one with $\sigma_B / B = 1~\text{ppm}$ and one with $\sigma_B / B = 50~\text{ppb}$.
Two different exposure scenarios are also plotted, one with $10^{-3}~\text{m}^3~\text{yr}$ and one with $10~\text{m}^3 \times 10~\text{yr}$.
These are all plotted as a function of density.

In [ ]:
# Generate number densities between 10^12 m^-3 and 10^20 m^-3
numberDensities = np.logspace(14, 20, 100)
# Define two different molecular tritium contaminations
hiMolTFrac = 1e-2
loMolTFrac = 1e-4

volumeBig = 10.0 # m^3
volumeSmall = 1e-3 # m^3
timeLong = 10.0 * year # 10 year run time in seconds
timeShort = 1.0 * year # 1 year run time in seconds
desiredRecoEff = 0.05
bField = 0.7 # Tesla
minTrappedPitchAngle = 89.0 # degrees
deltaTheta = 90.0 - minTrappedPitchAngle # degrees
eta = np.sin(np.radians(deltaTheta)) # Unitless
trapDepth = (bField / np.cos(np.arcsin(eta))**2) - bField # Tesla

# Calculate the required observation times for the desired efficiency
tObs = meanFreeTime(numberDensities, electronSpeed) * -np.log(desiredRecoEff)

# Look at large exposure with the low molecular tritium contamination and a low
# magnetic field uncertainty
sigmaM2_LargeExp_HiMolT_LowFieldUnc = experimentSigmaMSq_molT(volumeBig, numberDensities, bField, 50.0 / 1e9 * bField, detSampleRate, 
                                                              TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                              hiMolTFrac)
sigmaM2_LargeExp_LowMolT_LowFieldUnc = experimentSigmaMSq_molT(volumeBig, numberDensities, bField, 50.0 / 1e9 * bField, detSampleRate, 
                                                               TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                               loMolTFrac)

# Look at large exposure with the large magnetic field uncertainty
sigmaM2_LargeExp_LowMolT_HighFieldUnc = experimentSigmaMSq_molT(volumeBig, numberDensities, bField, 1e-6 * bField, detSampleRate, 
                                                                TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                                loMolTFrac)
sigmaM2_LargeExp_HiMolT_HighFieldUnc = experimentSigmaMSq_molT(volumeBig, numberDensities, bField, 1e-6 * bField, detSampleRate, 
                                                               TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                               hiMolTFrac)                                                                

# Now do the small exposures with the bad magnetic field uncertainty
sigmaM2_SmallExp_HiMolT_HighFieldUnc = experimentSigmaMSq_molT(volumeSmall, numberDensities, bField, 1e-6 * bField, detSampleRate, 
                                                                TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                                hiMolTFrac)
sigmaM2_SmallExp_LowMolT_HighFieldUnc = experimentSigmaMSq_molT(volumeSmall, numberDensities, bField, 1e-6 * bField, detSampleRate, 
                                                                TNoise5K, deltaV, trapDepth, tObs, timeLong, bkgRate, epsilonP, 
                                                                loMolTFrac)

# Plot the result
plt.figure()
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_LargeExp_HiMolT_LowFieldUnc), 
           label='$\\rm{T}_2 = 10^{-2}$, $\\frac{\sigma_B}{B} = 50 \\times 10^{-9}$',
           linestyle='--', color='b')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_LargeExp_LowMolT_LowFieldUnc), 
           label='$\\rm{T}_2 = 10^{-4}$, $\\frac{\sigma_B}{B} = 50 \\times 10^{-9}$',
           linestyle='-', color='b')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_LargeExp_HiMolT_HighFieldUnc), 
           label='$\\rm{T}_2 = 10^{-2}$, $\\frac{\sigma_B}{B} = 10^{-6}$',
           color='r', linestyle='--')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_LargeExp_LowMolT_HighFieldUnc), 
           label='$\\rm{T}_2 = 10^{-4}$, $\\frac{\sigma_B}{B} = 10^{-6}$',
           color='r')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_SmallExp_HiMolT_HighFieldUnc), 
           label='$\\rm{T}_2 = 10^{-2}$, $\\frac{\sigma_B}{B} = 10^{-6}$',
           color='g', linestyle='--')
plt.loglog(numberDensities / 1e6, np.sqrt(1.28 * sigmaM2_SmallExp_LowMolT_HighFieldUnc), 
           label='$\\rm{T}_2 = 10^{-4}$, $\\frac{\sigma_B}{B} = 10^{-6}$',
           color='g')
plt.xlabel('Number density [cm$^{-3}$]')
plt.ylabel('90\% CL sensitivity on $m_{\\beta}$ [eV]')
plt.grid()
plt.legend()
# Set y axis limits
plt.ylim([5e-3, 100])
plt.show()